# ETL for extracting data from the files of B3 - historical data


By SRS - 2S

V.1

Data extraction from the B3 by site and all files must be downloaded directly:


https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/


Objective:  Extract information from the downloaded from B3 on historical quotes and save in format .csv

In [21]:
# Importing libraries
import pandas as pd


In [22]:
# Setting up the B3 file directory
colab_folder = 'B3/'

if colab_folder: # Authorise Colab com by copying the token generated from the link

    from google.colab import drive # Important method for working with Google Drive folders
    drive.mount('/content/gdrive') # Mount and make available the file system in Colab (linux)

    # Navigate to your files folder
    %cd gdrive/MyDrive/{colab_folder}
    ! ls #  Linux command to list the contents



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/MyDrive/B3/'
/content/gdrive/MyDrive/B3
B3_2020_a_2024.csv  COTAHIST_A1993.TXT	COTAHIST_A2001.TXT  COTAHIST_A2009.TXT	COTAHIST_A2017.TXT
COTAHIST_A1986.TXT  COTAHIST_A1994.TXT	COTAHIST_A2002.TXT  COTAHIST_A2010.TXT	COTAHIST_A2018.TXT
COTAHIST_A1987.TXT  COTAHIST_A1995.TXT	COTAHIST_A2003.TXT  COTAHIST_A2011.TXT	COTAHIST_A2019.TXT
COTAHIST_A1988.TXT  COTAHIST_A1996.TXT	COTAHIST_A2004.TXT  COTAHIST_A2012.TXT	COTAHIST_A2020.TXT
COTAHIST_A1989.TXT  COTAHIST_A1997.TXT	COTAHIST_A2005.TXT  COTAHIST_A2013.TXT	COTAHIST_A2021.TXT
COTAHIST_A1990.TXT  COTAHIST_A1998.TXT	COTAHIST_A2006.TXT  COTAHIST_A2014.TXT	COTAHIST_A2022.TXT
COTAHIST_A1991.TXT  COTAHIST_A1999.TXT	COTAHIST_A2007.TXT  COTAHIST_A2015.TXT	COTAHIST_A2023.TXT
COTAHIST_A1992.TXT  COTAHIST_A2000.TXT	COTAHIST_A2008.TXT  COTAHIST_A2016.TXT	COTAHIST_A2024.TXT


In [30]:
# Function for extracting data from files downloaded from the B3 website (historical data)
def read_file(path, name_file, year_date, type_file):

  _file = f'{path}{name_file}{year_date}.{type_file}'

  # Define column widths
  colspecs = [(2, 10),  (11, 12), (13, 24), (28, 39),
            (57, 69), (70, 82), (83, 95), (109, 121),
            (153, 170), (171, 188)]

  # Column names
  names = ['xtrading_date', 'codbdi', 'action', 'name_action', 'open_price',
         'maximum_price', 'minimum_price', 'closing_price', 'trading_quantity',
         'trade_volume']

  # Path of the positional TXT file
  #path_files = r'C:\downloads\B3\COTAHIST_A2023.txt'

  # Reading the positional TXT file
  df = pd.read_fwf(_file, colspecs=colspecs, skiprows=1, skipfooter=1, names=names)

  return df


In [31]:
# Selecting only the type of share = Standard Lot (2)
def filter_stocks(df):
  df = df[df['codbdi'] == 2]
  df = df.drop(['codbdi'], axis = 1)
  df.head()

  return df


In [32]:
# Date field conversion
def parse_date(df):

  df['trading_date'] = pd.to_datetime(df['xtrading_date'], format='%Y%m%d')

  return(df)


In [33]:
# version of numeric fields
def parse_values(df):

  df['open_price']    = (df['open_price']    / 100 ).astype(float)
  df['maximum_price'] = (df['maximum_price'] / 100 ).astype(float)
  df['minimum_price'] = (df['minimum_price'] / 100 ).astype(float)
  df['closing_price'] = (df['closing_price'] / 100 ).astype(float)

  return(df)


In [36]:
# Combining the files

def concat_files(path, name_file, year_date, type_file, final_file):


  for i, y in enumerate(year_date):
    df = read_file(path, name_file, y, type_file)
    df = filter_stocks(df)
    df = parse_date(df)
    df = parse_values(df)

    if i == 0:
      df_final = df
    else:
      df_final = pd.concat([df_final, df])

  df_final = df.drop('xtrading_date', axis=1)
  df_final.to_csv(f'{path}//{final_file}', index=False)

In [37]:
#  Performing primary data extraction and conversion - last 5 years
year_date = [ '2020', '2021', '2022', '2023', '2024']

path = f'/content/gdrive/MyDrive/B3/'
name_file = 'COTAHIST_A'
type_file = 'TXT'
final_file = 'B3_2020_a_2024.csv'

concat_files( path, name_file, year_date, type_file, final_file)



In [38]:
# Open the new file
df_work = pd.read_csv('B3_2020_a_2024.csv')

# Show data
df_work.head()


,action,name_action,open_price,maximum_price,minimum_price,closing_price,trading_quantity,trade_volume,trading_date
0,ALR3,LLIAR,10.20,10.36,8.50,8.50,430400,401487500,2024-01-02
1,BCB4,BC BRASIL,23.98,23.98,22.79,22.88,1944700,4494730800,2024-01-02
2,BEV3,MBEV S/A,13.72,13.73,13.59,13.71,11690200,15983911100,2024-01-02
3,BDC3,RADESCO,15.26,15.26,14.98,15.11,4545600,6857685400,2024-01-02
4,LPA3,LPARGATAS,10.11,10.30,9.99,10.00,2900,2912000,2024-01-02
